In [1]:
!sh run.sh

Copying observations from the provided folders...1183. Done!.


In [2]:
from multiprocessing import Pool
import pandas as pd
from config import instruments_list, instrument_pitch_bondaries, shift_step, \
    columns_with_shift_meta as template_names, \
    re_instrumented_chunked_copies_columns_with_shift_meta as names, \
    csv_paths_with_shift_meta as template_csv_path
import warnings

warnings.simplefilter("ignore")

In [3]:
dispatcher_df = pd.read_csv(f"{template_csv_path}.csv", names=template_names)

In [4]:
list_of_instrument_specific_frames = []
for instrument in instruments_list:
    boundaries = instrument_pitch_bondaries[instrument]
    observations_in_range = dispatcher_df.loc[(dispatcher_df["max_note"] <= boundaries["max_note"]) &
                                              (dispatcher_df["min_note"] >= boundaries["min_note"])]
    observations_in_range["instrument"] = instrument
    list_of_instrument_specific_frames.append(observations_in_range)
re_intrumented = pd.concat(list_of_instrument_specific_frames, ignore_index=True)    

In [5]:
boundaries = pd.DataFrame.from_records(
        [instrument_pitch_bondaries[instrument]
         for instrument in re_intrumented["instrument"].values])[["max_note", "min_note"]]
re_intrumented["up_boundary"] = boundaries["max_note"].values
re_intrumented["down_boundary"] = boundaries["min_note"].values

In [6]:
list_of_shift_specific_frames = []
for shift in range(-127, 127):
    criterion_results = \
        (re_intrumented["max_note"] + shift_step * shift <= re_intrumented["up_boundary"]) & \
        (re_intrumented["min_note"] + shift_step * shift >= re_intrumented["down_boundary"])
    observations_in_range_for_current_shift = re_intrumented.loc[criterion_results]
    observations_in_range_for_current_shift["relative_shift"] = shift
    list_of_shift_specific_frames.append(observations_in_range_for_current_shift)
df = pd.concat(list_of_shift_specific_frames, ignore_index=True)

In [7]:
df["shift"] = df["relative_shift"] + df["min_note"]//shift_step
df["shift"] -= df["shift"].min()

absolute_shift_range_deltas = [4, -10]
valid_names = list(set(df.loc[df["shift"]==df["shift"].min()+absolute_shift_range_deltas[0], "name"].unique())\
                   .intersection(df.loc[df["shift"]==df["shift"].max()+absolute_shift_range_deltas[1], "name"].unique()))
outcast_criterion = df["name"] != valid_names[0]
for valid_name in valid_names[1:]:
    outcast_criterion = outcast_criterion & (df["name"] != valid_name)
elite_df = df.loc[(outcast_criterion == False) & (df["shift"] <= df["shift"].max() + absolute_shift_range_deltas[1])\
                  & (df["shift"]>=df["shift"].min()+absolute_shift_range_deltas[0])]
print(f'names left in line: {len(elite_df["name"].value_counts().index.values)}, while number of ' 
      f'observations is {len(elite_df)}')


names left in line: 172, while number of observations is 23736


In [ ]:
if __name__ == '__main__':
    print(4)

In [8]:
from multiprocessing import Pool
import pandas as pd
from config import re_instrumented_chunked_copies_csv_path as target_csv_path, \
                   re_instrumented_chunked_copies_storage_path as target_storage_path,\
                   re_instrumented_chunked_copies_columns_with_shift_meta as names
import augmentations
if __name__ == '__main__':
    print("Name is indeed main.")
    target_storage_path.mkdir(parents=True, exist_ok=True)
    for instrument in instruments_list:
        paths = elite_df.loc[(elite_df["instrument"] == instrument), "path"].value_counts().index.values 

        if len(paths) == 0:
            print("no observation in range")
            continue

        def f(x):
            augmentations.change_instrument(f"{x}.mid",
                                            f"{target_storage_path}/{x.split('/')[-1]}_{instrument}.mid", instrument)
        with Pool(5) as p:
            p.map(f, paths)
elite_df["path"] = [f"{target_storage_path}/{path.split('/')[-1]}_{instrument}" 
                    for path, instrument in elite_df[["path", "instrument"]].values]

Name is indeed main.


In [9]:
from multiprocessing import Pool
import pandas as pd
from config import instrument_pitch_bondaries, shift_step, \
    re_instrumented_chunked_copies_columns_with_shift_meta as template_names, \
    shifted_re_instrumented_chunked_copies_columns_with_shift_meta as names, \
    re_instrumented_chunked_copies_csv_path as template_csv_path, \
    shifted_re_instrumented_chunked_copies_csv_path as target_csv_path, \
    shifted_re_instrumented_chunked_copies_storage_path as target_storage_path
import augmentations

import warnings

warnings.simplefilter("ignore")

if __name__ == '__main__':
    print("Name is indeed main.")
    target_storage_path.mkdir(parents=True, exist_ok=True)
    
    for shift in range(-127 // shift_step, 127 // shift_step):
        paths = elite_df.loc[(elite_df["relative_shift"] == shift), "path"].value_counts().index.values 
        if len(paths) == 0:
            continue

        def f(path):
            augmentations.change_octave(f"{path}.mid",
                                        f"{target_storage_path}/{path.split('/')[-1]}_{shift}.mid", shift, shift_step)


        with Pool(5) as p:
            p.map(f, paths)

elite_df["path"] = [f"{target_storage_path}/{path.split('/')[-1]}_{shift}"
                    for path, shift in elite_df[["path", "relative_shift"]].values]
elite_df[names].to_csv(f"{target_csv_path}.csv", index=False, header=False, mode="a")

Name is indeed main.


In [ ]:
from config import ROOT
elite_df[names].to_csv(f"{ROOT}/elite.csv", index=False, header=True, mode="a")